# Libraries

In [1]:
import numpy as np 
import tensorflow as tf 
import gym 
from gym import wrappers
import pygame 
from gym.utils import play # manual play 
import matplotlib.pyplot as plt 
import seaborn as sns 
import io
import base64
from IPython.display import HTML

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


# Constants 

In [2]:
ATARI_GAME = 'Breakout-v4'
NUM_OF_EPISODES = 10 

# Open AI Gym Environment Setup

In [3]:
env = gym.make(ATARI_GAME)
env = wrappers.Monitor(env, "./gym-results", force=True) # Cache game

# # Uncomment line below to play the game as a human
# #play.play(env, zoom=3)

# Let the environment play, by taking random actions at each timestep
for episode in range(NUM_OF_EPISODES): 
    env.reset()
    while True: 
        random_action = env.action_space.sample() 
        observation, reward, is_done, info = env.step(random_action)
        if is_done:
            break 
env.close() 

# Display offline mp4 simulation
Source: https://kyso.io/eoin/openai-gym-jupyter?utm_campaign=News&utm_medium=Community&utm_source=DataCamp.com

In [4]:
video = io.open('./gym-results/openaigym.video.%s.video000000.mp4' % env.file_infix, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
    <video width="360" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode('ascii')))